In [1]:
import os
import numpy as np
from pydub import AudioSegment
import wave
import struct
def load_mp3(filename):
    '''
    输入mp3文件的名，返回特征数据
    '''
    MP3_File = AudioSegment.from_mp3(file=filename)
    MP3_File.export('shiyan.wav', format="wav")#生成wave文件
    #wav文件读取
    '''
    nchannels:声道数,
    sampwidth：采样宽度,
    framerate：帧速率,
    nframes：帧的总量，
    '''
    f = wave.open('shiyan.wav','rb')
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    strData = f.readframes(nframes)#读取音频，字符串格式
    waveData = np.frombuffer(strData,dtype=np.int16)#将字符串转化为int
    waveData = waveData*1.0/(max(abs(waveData)))#wave幅值归一化
    temp_result = np.reshape(waveData,[nframes,nchannels])
    f.close()
    nframes=5000000
    result=temp_result[:nframes]
    outData = result#待写入wav的数据，这里仍然取waveData数据
    outData=np.expand_dims(outData,axis=0)
    outData = tf.cast(outData, dtype=tf.float32)
    return outData

def make_songs(result):
    test=np.array(result)
    nframes=5000000
    nchannels=2
    waveData=test[:nframes]
    outData = waveData
    outData = np.reshape(outData,[nframes*nchannels,1])
    outfile = 'outy.wav'
    outwave = wave.open(outfile, 'wb')#定义存储路径以及文件名
    sampwidth = 2
    fs = 44100
    data_size = len(outData)
    framerate = int(fs)
    nframes = data_size
    comptype = "NONE"
    compname = "not compressed"
    outwave.setparams((nchannels, sampwidth, framerate, nframes,
        comptype, compname))
    outData=outData
    for i in range(len(outData)):
        v=outData[i]
        outwave.writeframes(struct.pack('h', int(v * 64000 / 2)))#outData:16位，-32767~32767，注意不要溢出
    outwave.close()

In [2]:
import tensorflow as tf
class DCSGAN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1=tf.keras.layers.Dense(20, activation='relu')
        self.conv1=tf.keras.layers.Conv1D(filters=10,kernel_size=200,padding='same')
        self.dense2=tf.keras.layers.Dense(200000,activation='tanh')
        self.trconv1=tf.keras.layers.Conv1DTranspose(filters=10,kernel_size=20)
        self.conv2=tf.keras.layers.Conv1D(filters=50,kernel_size=2)
        self.reshape1=tf.keras.layers.Reshape(target_shape=(10,5000000))
        self.reshape2=tf.keras.layers.Reshape(target_shape=(200000,10))
        self.flat1=tf.keras.layers.Flatten()
    # def generate(self,inputs):

    def call(self,inputs):
        x=self.conv1(inputs)
        x=self.reshape1(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.reshape2(x)
        x=self.trconv1(x)
        x=self.conv2(x)
        x=self.flat1(x)
        return x

In [3]:
train=load_mp3('63892.mp3')
new_train=[0 for i in range(900)]
temp=np.array(train).flatten()
result=list(temp)+new_train
result=np.array(result)
result=tf.cast(result, dtype=tf.float32)
result=tf.reshape(result,(1,10000900))

In [4]:
train1=np.random.rand(1,5000000,2)
train1=tf.cast(train1,dtype=tf.float32)
model=DCSGAN()
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
for i in range(1000):
    with tf.GradientTape() as tape:
        test=model(train1)
        loss=tf.reduce_mean(tf.square(result - test))
    print(loss)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads,model.trainable_variables))

tf.Tensor(0.029441312, shape=(), dtype=float32)
tf.Tensor(0.02943537, shape=(), dtype=float32)


ResourceExhaustedError: OOM when allocating tensor with shape[5000000,20] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:MatMul]

In [5]:
new_test=np.array(test)
new_test=new_test[0][:10000000]
new_test=tf.cast(new_test,dtype=tf.float32)
new_test=tf.reshape(new_test,(1,5000000,2))
new_test.shape

In [ ]:
make_songs(new_test)

In [6]:
import tensorflow as tf;
print(tf.__version__)

2.4.1
